# Projeto 1 - Ciência dos Dados

Nome: Andressa Silva de Oliveira

Nome: Luana Mitsudo Coelho

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
# Importando as bibliotecas
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import functools
import operator
import re
import emoji

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'xiaomi.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo xiaomi.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

Treinamento  Valor
0               @miuirom miui 12 no rn8 xiaomi cade?      1
1  rt @dani13_f: meu cell note 8 xiaomi já tá com...      1
2  @notfakeholmes @brenomarquesof @arcanjobeni 3 ...      1
3     @juliaaacras o da xiaomi é excelente, sem mais      1
4  @rrlleal00 xiaomi mi 9 se. comprei na amazon e...      0

In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

Teste  Valor
0  vcs acham msm que a câmera d xiaomi é melhor q...      1
1  pq as autorizadas do xiaomi só são no toba do ...      1
2  🔥 em stock! smart tvs xiaomi ao preço que quer...      0
3  xiaomi poco x3 é um verdadeiro sucesso de vend...      1
4  quero refazer pq na época eu usava o iphone 7 ...      1

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Escolhemos a marca Xiaomi e consideramos, para fins de análise, que os tweets relevantes são aqueles que elogiam de alguma forma, pedem melhorias ou criticam a marca e seus produtos. Na categoria não relevantes foram classificados os tweets de propagandas de lojas e aqueles que mencionam o nome da marca mas não se encaixam nas categorias já mencionadas.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
# Código para limpeza dos dados
# https://docs.python.org/3/library/re.html#
import re 
def cleanup(text):
    punctuation = '[!-.:?;]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [6]:
# Limpando a base de dados para Treinamento
treinamento_limpo = cleanup(" ".join(train.Treinamento).lower())
em = treinamento_limpo
em_split_emoji = emoji.get_emoji_regexp().split(em)
em_split_whitespace = [substr.split() for substr in em_split_emoji]
palavras_train = functools.reduce(operator.concat, em_split_whitespace)

In [7]:
# Convertendo treinamento em pd.Series
serie_train = pd.Series(palavras_train)
serie_train

0                    @miuirom
1                        miui
2                          12
3                          no
4                         rn8
                ...          
7263                      g5s
7264                   xiaomi
7265                       mi
7266                       a2
7267    https//tco/gtqmqfuxso
Length: 7268, dtype: object

In [8]:
# Tabela absoluta de treinamento
tabela_train_absoluta = serie_train.value_counts()
tabela_train_absoluta.head()

xiaomi    361
o         190
e         156
de        156
que       155
dtype: int64

In [9]:
# Tabela relativa de treinamento
tabela_train_relativa = serie_train.value_counts(True)
tabela_train_relativa.head()

xiaomi    0.049670
o         0.026142
e         0.021464
de        0.021464
que       0.021326
dtype: float64

In [10]:
# Separação de tweets em relevantes e irrelevantes
dados_R = train.loc[train.Valor == 1,:]
dados_I = train.loc[train.Valor == 0,:]

In [11]:
# Organizando a base de dados separada em relevantes e irrelevantes
dados_R_limpo = cleanup(" ".join(dados_R.Treinamento).lower())
dados_I_limpo = cleanup(" ".join(dados_I.Treinamento).lower())

In [12]:
# Separação de palavras pertencentes a twittes relevantes e irrelevantes
palavras_dados_R = dados_R_limpo.split()
palavras_dados_I = dados_I_limpo.split()

In [13]:
# Probabilidade de um tweet ser relevante ou irrelevante
prob_relev = train.Valor.value_counts(True, sort=False)
P_R = prob_relev[1]
P_I = prob_relev[0]
print(P_R)
print(P_I)

0.41456582633053224
0.5854341736694678


In [14]:
# Tabela de frequências para palavras relevantes
tabela_R = pd.Series(palavras_dados_R).value_counts()
tabela_R.head()

xiaomi    152
o         103
redmi      84
e          84
a          84
dtype: int64

In [15]:
# Tabela de frequências para palavras irrelevantes
tabela_I = pd.Series(palavras_dados_I).value_counts()
tabela_I.head()

xiaomi    209
o          87
que        76
de         74
e          72
dtype: int64

In [16]:
# Total de palavras relevantes
total_R = tabela_R.sum()
total_R

3572

In [17]:
# Total de palavras irrelevantes
total_I = tabela_I.sum()
total_I

3604

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [20]:
# Limpando a base de dados para teste e armazenando em uma nova coluna
cont = 0
for a in test.Teste:
    teste_limpo = cleanup(a).lower()
    test.loc[cont,'Limpos'] = teste_limpo
    cont += 1

In [21]:
# Verificando a coluna nova
test.head()

Teste  Valor  \
0  vcs acham msm que a câmera d xiaomi é melhor q...      1   
1  pq as autorizadas do xiaomi só são no toba do ...      1   
2  🔥 em stock! smart tvs xiaomi ao preço que quer...      0   
3  xiaomi poco x3 é um verdadeiro sucesso de vend...      1   
4  quero refazer pq na época eu usava o iphone 7 ...      1   

                                              Limpos  
0  vcs acham msm que a câmera d xiaomi é melhor q...  
1  pq as autorizadas do xiaomi só são no toba do ...  
2  🔥 em stock smart tvs xiaomi ao preço que queri...  
3  xiaomi poco x3 é um verdadeiro sucesso de vend...  
4  quero refazer pq na época eu usava o iphone 7 ...

In [22]:
# Verificando o tipo de dados da coluna teste
type(test.Teste)

pandas.core.series.Series

In [23]:
# Separando os tweets em listas de palavras
lista_palavras_separadas = []
for frase in test.Limpos:
    em = frase
    em_split_emoji = emoji.get_emoji_regexp().split(em)
    em_split_whitespace = [substr.split() for substr in em_split_emoji]
    palavras_test = functools.reduce(operator.concat, em_split_whitespace)
    lista_palavras_separadas.append(palavras_test)

In [54]:
# Calculando as probabilidades de relevância em um tweet a partir do classificador desenvolvido
i = 0
a = 0
while i < len(lista_palavras_separadas):
    Prob_frase_R = 1
    Prob_frase_I = 1
    
    for a in (lista_palavras_separadas[i]):
        if a not in tabela_R:
            Prob_frase_R *= ((0+1)/(total_R + (total_R+total_I)))

        else:
            Prob_frase_R *= (tabela_R[a] + 1)/(total_R + (total_R+total_I))
            
        if a not in tabela_I:
            Prob_frase_I *= ((0+1)/(total_R + (total_R+total_I)))

        else:
            Prob_frase_I *= ((tabela_I[a] +1)/(total_R + (total_R+total_I)))
            
    if (Prob_frase_R*P_R) > (Prob_frase_I*P_I):
        test.loc[i,'Classificador'] = 1
    else:
        test.loc[i,'Classificador'] = 0
    i += 1

In [55]:
# Tabela comparativa da classificação do modelo com a nossa
tab = pd.crosstab(test['Classificador'],test['Valor'])
tab

Valor           0   1
Classificador        
0.0            82  25
1.0            37  55

In [56]:
# Parâmetros obtidos a partir da tabela - Porcentagens calculadas em relação ao total de tweets dados pelo classificador como relevantes ou irrelevantes 

Porc_R_classif_R = (tab[1][1]/(tab[0][1]+tab[1][1]))
Porc_I_classif_R = (tab[0][1]/(tab[0][1]+tab[1][1]))
Porc_I_classif_I = (tab[0][0]/(tab[0][0]+tab[1][0]))
Porc_R_classif_I = (tab[1][0]/(tab[1][0]+tab[0][0]))

print(f'Porcentagem de verdadeiros positivos = {Porc_R_classif_R}')
print(f'Porcentagem de falsos positivos = {Porc_I_classif_R}')
print(f'Porcentagem de verdadeiros negativos = {Porc_I_classif_I}')  
print(f'Porcentagem de falsos negativos = {Porc_R_classif_I}')

Porcentagem de verdadeiros positivos = 0.5978260869565217
Porcentagem de falsos positivos = 0.40217391304347827
Porcentagem de verdadeiros negativos = 0.7663551401869159
Porcentagem de falsos negativos = 0.2336448598130841


In [57]:
# Parâmetros comparativos sobre a performance do classificador
relacao_vp_fp = (Porc_R_classif_R - Porc_I_classif_R)*100
relacao_vn_fn = (Porc_I_classif_I - Porc_R_classif_I)*100

print(f'Diferença entre verdadeiros positivos e falsos positivos, em porcentagem = {relacao_vp_fp}')
print(f'Diferença entre verdadeiros negativos e falsos negativos, em porcentagem = {relacao_vn_fn}')

Diferença entre verdadeiros positivos e falsos positivos, em porcentagem = 19.565217391304344
Diferença entre verdadeiros negativos e falsos negativos, em porcentagem = 53.271028037383175


___
### Concluindo

#Comparativo qualitativo sobre os percentuais obtidos para discussão da performance do classificador.

    Com base nas porcentagens obtidas acima, percebe-se que o classificador, ainda em seu primeiro MVP, 
    apresenta uma performance satisfatória, fornecendo analises de qualidade sobre os dados coletados, 
    uma vez que os valores obtidos para porcentagens de verdadeiros positivos e verdadeiros negativos 
    superam as porcentagens de falsos positivos e falsos negativos em, respectivamente, 19,56% e 53,27%.
    

#Explique como são tratadas as mensagens com dupla negação e sarcasmo.

   Para a análise de mensagens com dupla negação, tomou-se, como exemplo, a seguinte frase da base de teste: "vcs acham msm 
que a câmera d xiaomi é melhor que iphone que coragy não chega nem fodendo pode ter quantos mp quiser fii". Nesse caso, percebeu-se que a frase foi rotulada, pelo classificador, como relevante, recebendo valor 1, o mesmo atribuído manualmente na etapa inicial do projeto.
    Já para a análise de mensagens com sarcasmo, toma-se como exemplo o seguinte tweet: "atualização sobre meus xiaomi airdots bateria agora dura 10 minutos". Nesse caso, o classificador atribuiu o valor 1, tendo considerado esse tweet como relevante, enquanto o valor atribuído manualmente na etapa inicial foi o de irrelevante.


#Proposta de plano de expansão. Por que nosso projeto deve continuar sendo financiado? 

    O classificador Naive-Bayes é uma ferramenta que tem sido amplamente utilizada para a análise de textos, 
    apresentando níveis de qualidade que se destacam com relação a outras ferramentas que possuem o mesmo 
    propósito. Nesse contexto,considerando-se a análise dos dados obtidos acerca do funcionamento do 
    classificador construído longo deste arquivo, pode-se perceber que, mesmo em sua primeira versão, 
    ele apresentou resultados satisfatórios, com uma assertividade maior do que 55% na avaliação dos tweets 
    como relevantes ou irrelevantes. Além disso, mediante a realização dos aprimoramentos pretendidos: aumentar 
    a base de dados  considerada para a construção do classificador, criar categorias intermediárias de relevância 
    considerando a probabilidade, considerar as relações discursivas entre pares de palavras e, inclusive, buscar 
    diminuir a suposição de independência entre as palavras por meio de procedimentos como o uso de métodos de 
    regressão, que  vêm sendo estudados, a performance tornaria-se ainda mais precisa e coerente, aumentando a 
    assertividade das análises fornecidas. Dessa forma, a continuidade do financiamento do projeto, traria grandes 
    lucros para a empresa de interesse em termos de compreensão das necessidades do usuário e de percepção de 
    possíveis melhorias para alavancar os seus produtos e a sua imagem no mercado. 

#Por que não se deve alimentar a base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.

    A base de treinamento não pode ser alimentada automaticamente com o uso do próprio classificador 
    uma vez que este aprende com os tweets já classificados manualmente como relevantes ou irrelevantes. 
    Dessa forma, se o classificador criasse novos tweets para sua base de treinamento, ele não estaria 
    aperfeiçoando a sua análise, mas sim aplicando aquilo que já aprendeu anteriomente, então não haveria 
    melhora na precisão das previsões.


#Propostas de diferentes cenários de uso para o classificador Naive-Bayes.

    Dentre os cenários em que se pode usar o classificador Naive-Bayes, têm-se:
    - Filtrar anúncios em sites de forma personalizada para cada usuário, classificando como relevantes aqueles que 
    anunciam produtos, sites e serviços que a pessoa usualmente busca;
    - Filtrar e-mails, classificando-os como spam, principais, etc. de acordo com o conteúdo (Por exemplo: se é 
    propaganda ou não), com o remetente (Por exemplo: se é um contato frequente ou uma conta comercial), dentre 
    outras variáveis;
    - Configurar um site a partir da atribuição de tópicos (Topic Tagging) aos textos nele presentes. Dessa forma, 
    um texto pode, automaticamente, receber algumas classificações (Por exemplo: Um site pode ter suas matérias 
    categorizadas como pertencentes aos tópicos "esporte", "cultura", "moda", dentre outros);
    - Detectar fraudes em transações financeiras, identificando se  são "legais" ou "suspeitas";
    - Classificar clientes como "alto", "médio" ou "baixo" risco para aprovação de crédito;
    - Na área de bioinformática, identificar a classe de proteínas e, consequentemente, possibilitar o descobrimento
    de suas funções, dentre outras aplicações.

FONTES: https://www.devmedia.com.br/naive-bayes-mineracao-de-dados-na-pratica-revista-sql-magazine-110/27490

#Sugestão e explicação de melhorias reais no classificador:

    Sugestões para melhoria na performance do nosso classificador:
    - Aumentar a base de dados considerada para a construção do classificador (aumentando o valor do parâmetro "n" no  
    arquivo jupyter "Projeto1_Obtenção_dos_tweets.ipynb", que define a quantia mínima de mensagens capturadas para a 
    base de treinamento, e classificando-os manualmente como relevantes ou irrelevantes). A partir dessa modificação, o 
    classificador terá mais dados nos quais se basear para fundamentar a classificação de futuros tweets, reduzindo a 
    taxa de erros nesse processo e melhorando a sua performance;
    - Criar categorias intermediárias de relevância considerando a probabilidade (Para tal, deve-se criar novas variáveis 
    que classificam cada faixa de valor, dada em probabilidade, em uma categoria. Ademais, deve-se adicionar essa 
    classificação a uma coluna nova do tipo categórica no Dataframe). Considerando essa adaptação, a probabilidade de 
    uma frase relevante, mesmo que em grau pequeno, ser classificada como irrelevante será reduzida;
    - Realizar o processo de treinamento e teste do classificador considerando as relações discursivas entre pares de 
    palavras (Como, por exemplo, "concessão", "contraste", etc.). Para isso, deve-se utilizar apenas as palavras das 
    classes substantivo e verbo uma vez que são marcadores discursivos e, portanto, indicam tais relações. Dessa forma, 
    considerar atributos mais representativos que predizem as relações supracitadas de forma mais eficiente em conjunto
    com o aumento da base de dados para treinamento pode aumentar ainda mais a eficiência do classificador, fazendo com 
    que o seu desempenho aumente. 
    
FONTES: https://sites.icmc.usp.br/taspardo/NILCTR0225-PardoNunes.pdf


___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis  OK
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis OK
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação 
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento OK
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto OK
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa) OK

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) 

https://www.devmedia.com.br/naive-bayes-mineracao-de-dados-na-pratica-revista-sql-magazine-110/27490

https://sites.icmc.usp.br/taspardo/NILCTR0225-PardoNunes.pdf

https://stackoverflow.com/questions/49921720/how-to-split-emoji-from-each-other-python

https://pypi.org/project/emoji/

https://www.cos.ufrj.br/~ines/enia07_html/pdf/28095.pdf

https://www.vooo.pro/insights/6-passos-faceis-para-aprender-o-algoritmo-naive-bayes-com-o-codigo-em-python/#:~:text=Dicas%20para%20melhorar%20o%20poder%20do%20modelo%20Naive%20Bayes,-Aqui%20v%C3%A3o%20algumas&text=Se%20o%20conjunto%20de%20dados,conjunto%20de%20dados%20de%20teste